In [1]:
using GeometryTypes: HyperRectangle, FlexibleConvexHull, Vec, contains
using CoordinateTransformations: AffineMap
using StaticArrays
using EnhancedGJK
using MeshCat
using Polyhedra
using StaticArrays: SVector
using GeometryTypes: HyperSphere, Point

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-12889552252254312941\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-12224369226189157124\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-12889552252254312941\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
geo1 = HyperRectangle{3,Float32}(Float32[-0.08559614, -0.125, 0.0], Float32[0.117596135, 0.25, 0.07049995])
geo2 = FlexibleConvexHull{Vec{3,Float32}}(Vec{3,Float32}[[-1.4459023, 0.8858041, 0.9851251], [-0.32686943, -0.23770937, -0.0762], [-0.32686943, -0.23770937, 0.204528], [-0.32686943, 0.23770937, -0.0762], [-0.32686943, 0.23770937, 0.204528], [0.32682246, -0.23770937, -0.0762], [0.32682246, -0.23770937, 0.204528], [0.32682246, 0.23770937, -0.0762], [0.32682246, 0.23770937, 0.204528]])

convex_hull_gg(vertices) = collect(points(vrep(polyhedron(hrep(polyhedron(vrep(vertices)))))))
geo2_ch = FlexibleConvexHull(map(Vec{3,Float32}, convex_hull_gg(map(Array, geo2.vertices))))
# geo2_ch = FlexibleConvexHull{Vec{3,Float32}}(Vec{3,Float32}[[-0.32686943, -0.23770937, 0.204528], [-0.32686943, -0.23770937, -0.0762], [-0.32686943, 0.23770937, -0.0762], [0.32682246, -0.23770937, 0.204528], [0.32682246, -0.23770937, -0.0762], [-1.4459023, 0.8858041, 0.9851251], [0.32682246, 0.23770937, -0.0762], [0.32682246, 0.23770937, 0.204528]])

definitely_inside__Fgeo2 = Vec{3, Float32}(-1.4459023, 0.8858041, 0.9851251)
tx__Fworld__Fgeo1 = AffineMap((@SArray Float32[1.0 -1.3267949e-6 1.7603859e-12; 1.3267949e-6 1.0 1.3267932e-6; -3.5207683e-12 -1.3267932e-6 1.0]), (@SArray Float32[0.28507775, 4.0943226e-7, 1.0193151]))
tx__Fworld__Fgeo2 = AffineMap((@SArray Float32[0.809017 -0.58778524 0.0; 0.58778524 0.809017 0.0; 0.0 0.0 1.0]), (@SArray Float32[2.0, 0.1, 0.0762]))

# Convex hull check
# result1 = EnhancedGJK.gjk(geo1, geo2, tx__Fworld__Fgeo1, tx__Fworld__Fgeo2)
# result2 = EnhancedGJK.gjk_original(geo1, geo2, tx__Fworld__Fgeo1, tx__Fworld__Fgeo2)

# Convex hull check
result1 = EnhancedGJK.gjk(geo1, geo2_ch, tx__Fworld__Fgeo1, tx__Fworld__Fgeo2)  
result2 = EnhancedGJK.gjk_original(geo1, geo2_ch, tx__Fworld__Fgeo1, tx__Fworld__Fgeo2)

print("Final weights: ", result1.weights,"\n")
print("Final simplex: ", result1.simplex,"\n")
print("Iterations: ", result1.iterations,"\n")  # terminates in 5 iterations
if result1.in_collision
    @show simplex_penetration_distance(result1)
else
    @show separation_distance(result1)
end

print("\n", "Final weights: ", result2.weights,"\n")
print("Final simplex: ", result2.simplex,"\n")
print("Iterations: ", result2.iterations,"\n")  # continues to iterate
@show separation_distance(result2)

print("Collision Checks: ", "\n")
definitely_inside_Fgeo1 = inv(tx__Fworld__Fgeo1)(tx__Fworld__Fgeo2(definitely_inside__Fgeo2))
print(contains(geo1, definitely_inside_Fgeo1),"\n")
print(result1.in_collision,"\n") # pass
print(result2.in_collision,"\n") # fail

Final weights: Float32[0.4918945, 0.1321623, 0.37011227, 0.0058309333]
Final simplex: SArray{Tuple{3},Float32,1,3}[[0.0075002015, -0.091750085, -0.04200983], [0.0075002015, -0.09174999, 0.028490186], [0.0074998736, 0.15825, 0.028489828], [-1.2787571, -0.22518134, 1.0898153]]
Iterations: 5
simplex_penetration_distance(result1) = 0.0067486716f0

Final weights: Float32[0.3670002, 0.0, 0.63299984, 0.0]
Final simplex: SArray{Tuple{3},Float32,1,3}[[0.0074998736, 0.15825, 0.028489828], [-0.11009592, -0.09175025, -0.04200983], [0.0075002015, -0.09174999, 0.028490186], [-1.2787571, -0.22518134, 1.0898153]]
Iterations: 100
separation_distance(result2) = 0.029460728f0
Collision Checks: 
true
true
false


In [3]:
function visualize_simplex(vis::Visualizer, simplex)
    p = polyhedron(vrep(simplex))
    setobject!(vis[:simplex], Polyhedra.Mesh(p))
    for (i, point) in enumerate(simplex)
        setobject!(vis["p$i"], HyperSphere(Point(point), convert(eltype(point), 0.03)))
    end
end

visualize_simplex (generic function with 1 method)

In [4]:
convex_mesh(points) = Polyhedra.Mesh(Polyhedra.polyhedron(Polyhedra.vrep(map(Array, points))))

vis = Visualizer()
# open(vis)
IJuliaCell(vis)

WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.IFrame}(WebIO.IFrame("<div\n    class=\"webio-mountpoint\"\n    data-webio-mountpoint=\"9170807062044742021\"\n>\n    <script>\n    if (window.require && require.defined && require.defined(\"nbextensions/webio-jupyter-notebook\")) {\n        console.log(\"Jupyter WebIO extension detected, not mounting.\");\n    } else if (window.WebIO) {\n        WebIO.mount(\n            document.querySelector('[data-webio-mountpoint=\"9170807062044742021\"]'),\n            {\"props\":{},\"nodeType\":\"Scope\",\"type\":\"node\",\"instanceArgs\":{\"imports\":{\"data\":[{\"name\":null,\"type\":\"js\",\"url\":\"\\/assetserver\\/cf544a313b8b4758c5b46eda729d77835c3c134e-main.min.js\"}],\"type\":\"async_block\"},\"id\":\"5700204874509158348\",\"handlers\":{\"meshcat-command\":[\"(function (val){return this.viewer.handle_command_message({\\\"data\\\":val})})\"]},\"systemjs_options\":null,\"mount_callbacks\":[\"function () {\\n    var handler = ((function (mc){var element=this.dom.children[0]; this.viewer=new mc.Viewer(element); WebIO.setval({\\\"name\\\":\\\"meshcat-request\\\",\\\"scope\\\":\\\"5700204874509158348\\\",\\\"id\\\":\\\"ob_02\\\",\\\"type\\\":\\\"observable\\\"},String(Date.now())); window.document.body.style.margin=\\\"0\\\"; return window.meshcat_viewer=this.viewer}));\\n    (WebIO.importBlock({\\\"data\\\":[{\\\"name\\\":null,\\\"type\\\":\\\"js\\\",\\\"url\\\":\\\"\\/assetserver\\/cf544a313b8b4758c5b46eda729d77835c3c134e-main.min.js\\\"}],\\\"type\\\":\\\"async_block\\\"})).then((imports) => handler.apply(this, imports));\\n}\\n\"],\"observables\":{\"meshcat-command\":{\"sync\":false,\"id\":\"ob_01\",\"value\":[]},\"meshcat-controls\":{\"sync\":true,\"id\":\"ob_03\",\"value\":[]},\"meshcat-request\":{\"sync\":true,\"id\":\"ob_02\",\"value\":\"\"}}},\"children\":[{\"props\":{\"className\":\"meshcat-viewer\",\"style\":{\"left\":0,\"right\":0,\"overflow\":\"hidden\",\"position\":\"absolute\",\"height\":\"100vh\",\"margin\":0,\"width\":\"100vw\"}},\"nodeType\":\"DOM\",\"type\":\"node\",\"instanceArgs\":{\"namespace\":\"html\",\"tag\":\"div\"},\"children\":[]}]},\n            window,\n        );\n    } else {\n        document\n            .querySelector('[data-webio-mountpoint=\"9170807062044742021\"]')\n            .innerHTML = (\n                '<strong>WebIO not detected. Please read ' +\n                '<a href=\"https://juliagizmos.github.io/WebIO.jl/latest/troubleshooting/not-detected/\">the troubleshooting guide</a> ' +\n                'for more information on how to resolve this issue.' +\n                '</strong>'\n            );\n    }\n    </script>\n</div>\n", "/assetserver/3b007542faf362b67669625b677317a876d878bb-generic-http.bundle.js"), Any[], Dict{Symbol,Any}())], Dict{Symbol,Any}(:style => Dict(:flexDirection => "column",:minHeight => "400px",:height => "100%",:margin => "inherit",:display => "flex",:width => "100%"))), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}(), Set(String[]), nothing, WebIO.Asset[], Dict{Any,Any}(), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(WebIO.AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f68867d5870, Task (runnable) @0x00007f68867d5870), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = ((function (){this.dom.style.height=\"100%\"; window.foo=this; return this.dom.children[0].children[0].style.flexGrow=\"1\"}));\n    (WebIO.importBlock({\"data\":[],\"type\":\"async_block\"})).then((imports) => handler.apply(this, imports));\n}\n")])

In [5]:
setobject!(vis[:geo1], geo1)
settransform!(vis[:geo1], tx__Fworld__Fgeo1)

setobject!(vis[:geo2], convex_mesh(geo2.vertices))
settransform!(vis[:geo2], tx__Fworld__Fgeo2)

# visualize_simplex(vis[:simplex], result1.simplex)

MeshCat Visualizer with path /meshcat/geo2